## Τεχνικές Λεπτομέρειες SVM

### Support Vector Machine (SVM) Παράμετροι:

**Kernel**: `linear`
- Χρησιμοποιείται γραμμικός πυρήνας που είναι αποδοτικός για high-dimensional δεδομένα κειμένου
- Κατάλληλος όταν η διάσταση των χαρακτηριστικών είναι μεγάλη σε σχέση με τον αριθμό δειγμάτων

**Probability**: `True`
- Ενεργοποιεί τον υπολογισμό πιθανοτήτων κλάσεων μέσω Platt scaling
- Χρήσιμο για την εξαγωγή confidence scores

**Random State**: `42`
- Εξασφαλίζει αναπαραγωγιμότητα των αποτελεσμάτων

### Διαδικασία Ταξινόμησης:

1. **Προεπεξεργασία**: Φόρτωση και καθαρισμός δεδομένων
2. **Feature Extraction**: Μετατροπή κειμένου σε αριθμητικά χαρακτηριστικά
3. **Train/Test Split**: Διαχωρισμός δεδομένων (80% εκπαίδευση, 20% αξιολόγηση)
4. **Model Training**: Εκπαίδευση SVM στα χαρακτηριστικά εκπαίδευσης
5. **Evaluation**: Αξιολόγηση με accuracy, precision, recall, f1-score

---

# Β1.i) Ταξινόμηση με SVM και Bag of Words / TF-IDF

Αυτό το notebook εκτελεί ταξινόμηση με Support Vector Machine χρησιμοποιώντας δύο διαφορετικές μεθόδους εξαγωγής χαρακτηριστικών:
1. Bag of Words (BoW) 
2. TF-IDF

Το dataset που χρησιμοποιείται είναι το "greek_legal_code" από το Hugging Face Hub.

In [ ]:
import sys
import os
import numpy as np
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset
import json

# Προσθήκη του parent directory στο path για να μπορούμε να εισάγουμε το utils
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import utils functions
try:
    from utils import (
        load_and_preprocess_data,
        run_experiment,
        script_execution_timer
    )
    print("Utils imported successfully")
except ImportError as e:
    print(f"Error importing utils: {e}")
    print("Continuing without utils functions...")

## Παραμετροποίηση

Ορισμός των παραμέτρων για τα πειράματα.

In [1]:
# Κοινές παράμετροι
DATASET_CONFIG = "subject"  # επιλογές: "volume", "chapter", "subject"
SUBSET_PERCENTAGE = 0.05     # 1.0 για όλα τα δεδομένα, <1.0 για υποσύνολο
N_SPLITS_CV = 1             # 1 για single split, >1 για K-Fold CV
RANDOM_STATE = 42
VECTORIZER_MAX_FEATURES = 5000
SINGLE_SPLIT_TEST_SIZE = 0.2

# Feature Engineering
SAVE_TRAINED_FEATURE_MODELS = True
LOAD_TRAINED_FEATURE_MODELS_IF_EXIST = True

print(f"Dataset config: {DATASET_CONFIG}")
print(f"Subset percentage: {SUBSET_PERCENTAGE}")
print(f"Max features: {VECTORIZER_MAX_FEATURES}")
print(f"Test size: {SINGLE_SPLIT_TEST_SIZE}")

Dataset config: subject
Subset percentage: 0.05
Max features: 5000
Test size: 0.2


## Φόρτωση και Προεπεξεργασία Δεδομένων

Φόρτωση του dataset και προετοιμασία για τα πειράματα.

In [2]:
# Φόρτωση των δεδομένων
print("Φόρτωση δεδομένων...")

# Αν υπάρχει η συνάρτηση utils, χρησιμοποιούμε αυτή
if 'load_and_preprocess_data' in globals():
    all_texts, all_labels, label_names, num_classes = load_and_preprocess_data(
        DATASET_CONFIG, SUBSET_PERCENTAGE, RANDOM_STATE
    )
else:
    # Εναλλακτική φόρτωση χωρίς utils
    print("Φόρτωση dataset χωρίς utils...")
    try:
        ds = load_dataset("AI-team-UoA/greek_legal_code", DATASET_CONFIG, trust_remote_code=True)
        dataset_split = ds['train']
        all_texts = dataset_split['text']
        all_labels = np.array(dataset_split['label'])
        label_names = dataset_split.features['label'].names
        num_classes = len(label_names)
        print(f"Dataset loaded successfully. Samples: {len(all_texts)}, Classes: {num_classes}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        all_texts, all_labels, label_names, num_classes = None, None, None, None

if all_texts is not None:
    print(f"Συνολικά samples: {len(all_texts)}")
    print(f"Αριθμός κλάσεων: {num_classes}")
    print(f"Ονόματα κλάσεων: {label_names}")

Φόρτωση δεδομένων...
Φόρτωση dataset χωρίς utils...
Error loading dataset: name 'load_dataset' is not defined


## 1. SVM με Bag of Words (BoW)

### Τεχνικά Χαρακτηριστικά Bag of Words:

**Bag of Words (BoW)** είναι μια μέθοδος εξαγωγής χαρακτηριστικών από κείμενο που:

- **Αναπαράσταση**: Μετατρέπει κάθε κείμενο σε διάνυσμα με διάσταση ίση με το μέγεθος του λεξιλογίου
- **Συχνότητα**: Κάθε στοιχείο του διανύσματος αντιπροσωπεύει τη συχνότητα εμφάνισης μιας λέξης
- **Απλότητα**: Αγνοεί τη σειρά των λέξεων και τη γραμματική δομή
- **Αραιότητα**: Παράγει αραιά διανύσματα (πολλά μηδενικά στοιχεία)
- **Max Features**: Χρησιμοποιούμε τις 5000 πιο συχνές λέξεις για να περιορίσουμε τη διάσταση

**Πλεονεκτήματα:**
- Απλή υλοποίηση και γρήγορη εκτέλεση
- Καλή απόδοση σε πολλές εφαρμογές ταξινόμησης κειμένου
- Δεν απαιτεί προεκπαίδευση

**Μειονεκτήματα:**
- Δεν καταγράφει τη σημασιολογική σχέση μεταξύ λέξεων
- Υψηλή διάσταση μπορεί να οδηγήσει σε curse of dimensionality

Εκτέλεση ταξινόμησης με SVM χρησιμοποιώντας Bag of Words για την εξαγωγή χαρακτηριστικών.

In [3]:
def run_svm_bow_experiment():
    """Εκτέλεση SVM με Bag of Words"""
    
    print("=== Ξεκινάει το πείραμα SVM με Bag of Words ===")
    start_time = time.time()
    
    # Παραμετροποίηση για BoW
    model_name_script = "SVM"
    feature_method_name = "BoW"
    base_run_id = f"{model_name_script}_{feature_method_name}_{DATASET_CONFIG}"
    
    # Ρύθμιση feature extraction
    feature_config = {
        'method': 'sklearn_vectorizer',
        'class': CountVectorizer,
        'params': {'max_features': VECTORIZER_MAX_FEATURES}
    }
    
    # Ρύθμιση μοντέλου
    model_class = SVC
    model_init_params = {'kernel': 'linear', 'probability': True, 'random_state': RANDOM_STATE}
    
    # Δημιουργία output directories
    script_dir = os.path.dirname(os.path.abspath(''))
    experiment_output_base_dir = os.path.join(script_dir, "outputs", base_run_id)
    reports_output_dir = os.path.join(experiment_output_base_dir, "reports")
    feature_models_output_dir = os.path.join(experiment_output_base_dir, "feature_models")
    
    # Δημιουργία directories αν δεν υπάρχουν
    os.makedirs(reports_output_dir, exist_ok=True)
    os.makedirs(feature_models_output_dir, exist_ok=True)
    
    print(f"Output directory: {experiment_output_base_dir}")
    
    # Αν υπάρχει η run_experiment από utils, χρησιμοποιούμε αυτή
    if 'run_experiment' in globals() and all_texts is not None:
        try:
            run_experiment(
                all_texts=all_texts,
                all_labels=all_labels,
                label_names=label_names,
                num_classes=num_classes,
                feature_config=feature_config,
                model_class=model_class,
                model_init_params=model_init_params,
                n_splits_cv=N_SPLITS_CV,
                single_split_test_size=SINGLE_SPLIT_TEST_SIZE,
                random_state=RANDOM_STATE,
                reports_output_dir=reports_output_dir,
                feature_models_output_dir=feature_models_output_dir,
                save_trained_feature_models=SAVE_TRAINED_FEATURE_MODELS,
                load_trained_feature_models_if_exist=LOAD_TRAINED_FEATURE_MODELS_IF_EXIST
            )
        except Exception as e:
            print(f"Error in run_experiment: {e}")
            print("Εκτέλεση απλουστευμένου πειράματος...")
            run_simple_experiment(feature_config, model_class, model_init_params, "BoW")
    else:
        print("Εκτέλεση απλουστευμένου πειράματος...")
        run_simple_experiment(feature_config, model_class, model_init_params, "BoW")
    
    end_time = time.time()
    print(f"=== Ολοκληρώθηκε το πείραμα BoW σε {end_time - start_time:.2f} δευτερόλεπτα ===")

# Εκτέλεση πειράματος BoW
if all_texts is not None:
    run_svm_bow_experiment()
else:
    print("Δεν μπορεί να εκτελεστεί το πείραμα - δεν φορτώθηκαν δεδομένα")

Δεν μπορεί να εκτελεστεί το πείραμα - δεν φορτώθηκαν δεδομένα


## 2. SVM με TF-IDF

### Τεχνικά Χαρακτηριστικά TF-IDF:

**TF-IDF (Term Frequency - Inverse Document Frequency)** είναι μια βελτιωμένη έκδοση του BoW που:

**Term Frequency (TF):**
- Μετρά τη συχνότητα μιας λέξης σε ένα έγγραφο
- TF(t,d) = (αριθμός εμφανίσεων του όρου t στο έγγραφο d) / (συνολικός αριθμός όρων στο έγγραφο d)

**Inverse Document Frequency (IDF):**
- Μετρά τη σπανιότητα μιας λέξης σε όλη τη συλλογή εγγράφων
- IDF(t,D) = log(N / |{d ∈ D : t ∈ d}|)
- Όπου N = συνολικός αριθμός εγγράφων

**TF-IDF Score:**
- TF-IDF(t,d,D) = TF(t,d) × IDF(t,D)
- Δίνει μεγαλύτερο βάρος σε λέξεις που είναι συχνές σε ένα έγγραφο αλλά σπάνιες στη συλλογή

**Πλεονεκτήματα:**
- Μειώνει τη σημασία των κοινών λέξεων (stop words)
- Αναδεικνύει τις χαρακτηριστικές λέξεις κάθε εγγράφου
- Καλύτερη αναπαράσταση της σημασίας των λέξεων

**Μειονεκτήματα:**
- Πιο υπολογιστικά πολύπλοκο από το BoW
- Εξακολουθεί να αγνοεί τη σειρά των λέξεων

**Παράμετροι sklearn.TfidfVectorizer:**
- `max_features=5000`: Χρήση των 5000 πιο σημαντικών όρων
- `smooth_idf=True`: Εξομάλυνση IDF για αποφυγή διαίρεσης με μηδέν
- `norm='l2'`: L2 κανονικοποίηση των διανυσμάτων

Εκτέλεση ταξινόμησης με SVM χρησιμοποιώντας TF-IDF για την εξαγωγή χαρακτηριστικών.

In [4]:
def run_svm_tfidf_experiment():
    """Εκτέλεση SVM με TF-IDF"""
    
    print("=== Ξεκινάει το πείραμα SVM με TF-IDF ===")
    start_time = time.time()
    
    # Παραμετροποίηση για TF-IDF
    model_name_script = "SVM"
    feature_method_name = "TF-IDF"
    base_run_id = f"{model_name_script}_{feature_method_name.replace('-', '')}_{DATASET_CONFIG}"
    
    # Ρύθμιση feature extraction
    feature_config = {
        'method': 'sklearn_vectorizer',
        'class': TfidfVectorizer,
        'params': {'max_features': VECTORIZER_MAX_FEATURES}
    }
    
    # Ρύθμιση μοντέλου
    model_class = SVC
    model_init_params = {'kernel': 'linear', 'probability': True, 'random_state': RANDOM_STATE}
    
    # Δημιουργία output directories
    script_dir = os.path.dirname(os.path.abspath(''))
    experiment_output_base_dir = os.path.join(script_dir, "outputs", base_run_id)
    reports_output_dir = os.path.join(experiment_output_base_dir, "reports")
    feature_models_output_dir = os.path.join(experiment_output_base_dir, "feature_models")
    
    # Δημιουργία directories αν δεν υπάρχουν
    os.makedirs(reports_output_dir, exist_ok=True)
    os.makedirs(feature_models_output_dir, exist_ok=True)
    
    print(f"Output directory: {experiment_output_base_dir}")
    
    # Αν υπάρχει η run_experiment από utils, χρησιμοποιούμε αυτή
    if 'run_experiment' in globals() and all_texts is not None:
        try:
            run_experiment(
                all_texts=all_texts,
                all_labels=all_labels,
                label_names=label_names,
                num_classes=num_classes,
                feature_config=feature_config,
                model_class=model_class,
                model_init_params=model_init_params,
                n_splits_cv=N_SPLITS_CV,
                single_split_test_size=SINGLE_SPLIT_TEST_SIZE,
                random_state=RANDOM_STATE,
                reports_output_dir=reports_output_dir,
                feature_models_output_dir=feature_models_output_dir,
                save_trained_feature_models=SAVE_TRAINED_FEATURE_MODELS,
                load_trained_feature_models_if_exist=LOAD_TRAINED_FEATURE_MODELS_IF_EXIST
            )
        except Exception as e:
            print(f"Error in run_experiment: {e}")
            print("Εκτέλεση απλουστευμένου πειράματος...")
            run_simple_experiment(feature_config, model_class, model_init_params, "TF-IDF")
    else:
        print("Εκτέλεση απλουστευμένου πειράματος...")
        run_simple_experiment(feature_config, model_class, model_init_params, "TF-IDF")
    
    end_time = time.time()
    print(f"=== Ολοκληρώθηκε το πείραμα TF-IDF σε {end_time - start_time:.2f} δευτερόλεπτα ===")

# Εκτέλεση πειράματος TF-IDF
if all_texts is not None:
    run_svm_tfidf_experiment()
else:
    print("Δεν μπορεί να εκτελεστεί το πείραμα - δεν φορτώθηκαν δεδομένα")

Δεν μπορεί να εκτελεστεί το πείραμα - δεν φορτώθηκαν δεδομένα


## Απλουστευμένη Εκτέλεση Πειραμάτων

Στην περίπτωση που δεν είναι διαθέσιμες οι συναρτήσεις utils, εκτελείται μια απλουστευμένη έκδοση.

In [ ]:
def run_simple_experiment(feature_config, model_class, model_init_params, method_name):
    """Απλουστευμένη εκτέλεση πειράματος χωρίς utils"""
    
    if all_texts is None or all_labels is None:
        print("Δεν είναι διαθέσιμα δεδομένα για το πείραμα")
        return
    
    print(f"Εκτέλεση απλουστευμένου πειράματος για {method_name}")
    
    # Διαχωρισμός σε training/testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        all_texts, all_labels, 
        test_size=SINGLE_SPLIT_TEST_SIZE, 
        random_state=RANDOM_STATE,
        stratify=all_labels
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    
    # Feature extraction
    vectorizer_class = feature_config['class']
    vectorizer_params = feature_config['params']
    vectorizer = vectorizer_class(**vectorizer_params)
    
    print(f"Εξαγωγή χαρακτηριστικών με {method_name}...")
    X_train_features = vectorizer.fit_transform(X_train)
    X_test_features = vectorizer.transform(X_test)
    
    print(f"Feature shape: {X_train_features.shape}")
    
    # Model training
    print(f"Εκπαίδευση SVM μοντέλου...")
    model = model_class(**model_init_params)
    model.fit(X_train_features, y_train)
    
    # Predictions
    print("Υπολογισμός προβλέψεων...")
    y_pred = model.predict(X_test_features)
    
    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=label_names)
    
    print(f"\n=== Αποτελέσματα για {method_name} ===")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(report)
    
    return {
        'accuracy': accuracy,
        'classification_report': report,
        'method': method_name
    }

print("Συνάρτηση απλουστευμένου πειράματος ορίστηκε επιτυχώς")

## Εντολές Εκτέλεσης

Για να εκτελέσετε τα πειράματα χωριστά μέσω terminal, χρησιμοποιήστε:

```bash
# Για SVM με Bag of Words
python merosB1/i/main_SVM_BoW.py

# Για SVM με TF-IDF
python merosB1/i/main_SVM_TFIDF.py
```

Τα αποτελέσματα θα αποθηκευτούν στους αντίστοιχους φακέλους `outputs/`.

In [ ]:
# Εναλλακτική εκτέλεση με υποσύνολο δεδομένων για γρήγορη δοκιμή
print("\n=== Γρήγορη Δοκιμή με Υποσύνολο Δεδομένων ===")

if all_texts is not None and len(all_texts) > 1000:
    # Χρήση 10% των δεδομένων για γρήγορη δοκιμή
    subset_size = int(len(all_texts) * 0.1)
    indices = np.random.choice(len(all_texts), subset_size, replace=False)
    
    subset_texts = [all_texts[i] for i in indices]
    subset_labels = all_labels[indices]
    
    print(f"Χρήση {subset_size} samples από {len(all_texts)} για γρήγορη δοκιμή")
    
    # Δοκιμή με BoW
    X_train, X_test, y_train, y_test = train_test_split(
        subset_texts, subset_labels, test_size=0.2, random_state=42, stratify=subset_labels
    )
    
    # BoW
    bow_vectorizer = CountVectorizer(max_features=1000)
    X_train_bow = bow_vectorizer.fit_transform(X_train)
    X_test_bow = bow_vectorizer.transform(X_test)
    
    svm_bow = SVC(kernel='linear', random_state=42)
    svm_bow.fit(X_train_bow, y_train)
    y_pred_bow = svm_bow.predict(X_test_bow)
    
    print(f"\nBoW Accuracy: {accuracy_score(y_test, y_pred_bow):.4f}")
    
    # TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    svm_tfidf = SVC(kernel='linear', random_state=42)
    svm_tfidf.fit(X_train_tfidf, y_train)
    y_pred_tfidf = svm_tfidf.predict(X_test_tfidf)
    
    print(f"TF-IDF Accuracy: {accuracy_score(y_test, y_pred_tfidf):.4f}")
    
    print("\nΣύγκριση αποτελεσμάτων:")
    print(f"TF-IDF vs BoW: {'+' if accuracy_score(y_test, y_pred_tfidf) > accuracy_score(y_test, y_pred_bow) else '-'}{abs(accuracy_score(y_test, y_pred_tfidf) - accuracy_score(y_test, y_pred_bow)):.4f}")
else:
    print("Δεν υπάρχουν αρκετά δεδομένα για δοκιμή")

## Αναμενόμενα Αποτελέσματα βάσει των προηγούμενων εκτελέσεων

Βάσει των αρχείων στο `outputsfinal/i/`, αναμένουμε:

- **TF-IDF εμφανίζει γενικά καλύτερη απόδοση** από το BoW
- **Καλύτερα αποτελέσματα** σε `subject` και `chapter` configurations
- **Χαμηλότερη απόδοση** σε `volume` configuration (περισσότερες κλάσεις)
- **Επάρκεια 5000 features** για ικανοποιητική απόδοση

Τα πειράματα σας θα επιβεβαιώσουν ή θα ανατρέψουν αυτές τις προσδοκίες!

## Σύγκριση των Μεθόδων

### Bag of Words vs TF-IDF

| Κριτήριο | Bag of Words | TF-IDF |
|----------|--------------|--------|
| **Υπολογιστική Πολυπλοκότητα** | Χαμηλή | Μέτρια |
| **Χειρισμός Κοινών Λέξεων** | Δεν φιλτράρει | Μειώνει τη σημασία τους |
| **Αναπαράσταση Σημασίας** | Απλή συχνότητα | Σταθμισμένη σημασία |
| **Μνήμη** | Χαμηλές απαιτήσεις | Μεσαίες απαιτήσεις |
| **Ευκολία Ερμηνείας** | Πολύ εύκολη | Μέτρια |
| **Απόδοση σε Μεγάλα Κείμενα** | Καλή | Πολύ καλή |

### Αναμενόμενα Αποτελέσματα:

**TF-IDF αναμένεται να υπερτερεί** γιατί:
- Μειώνει τη σημασία των stop words και κοινών λέξεων
- Αναδεικνύει χαρακτηριστικούς όρους κάθε νομικής κατηγορίας
- Καλύτερη αντιμετώπιση του vocabulary mismatch

**BoW μπορεί να είναι ανταγωνιστικό** σε περιπτώσεις:
- Όπου η συχνότητα εμφάνισης είναι σημαντική
- Μικρότερα datasets
- Όταν χρειαζόμαστε ταχύτητα

### Παράγονες που Επηρεάζουν την Απόδοση:

1. **Ποιότητα Dataset**: Καθαρότητα και ομοιογένεια των νομικών κειμένων
2. **Μέγεθος Λεξιλογίου**: 5000 max_features μπορεί να περιορίζει την έκφραση
3. **Κατανομή Κλάσεων**: Ισορροπία μεταξύ των νομικών κατηγοριών
4. **Γλωσσικά Χαρακτηριστικά**: Πλούσια ελληνική ορολογία του νομικού τομέα

---

## Σύνοψη Αποτελεσμάτων

Τα αποτελέσματα των πειραμάτων θα αποθηκευτούν στον φάκελο `outputs/` και θα περιλαμβάνουν:

1. **SVM με Bag of Words**: Classification reports και μετρικές απόδοσης
2. **SVM με TF-IDF**: Classification reports και μετρικές απόδοσης

Κάθε πείραμα παράγει:
- Classification report (σε μορφή JSON και text)
- Αποθηκευμένα μοντέλα feature extraction (εάν ενεργοποιηθεί)
- Λεπτομερή logs της εκτέλεσης

Τα αποτελέσματα μπορούν να συγκριθούν για να αξιολογηθεί η απόδοση των δύο μεθόδων εξαγωγής χαρακτηριστικών.

## Πρακτικές Συμβουλές για Βελτίωση

### Προεπεξεργασία Κειμένου:
- **Κανονικοποίηση**: Μετατροπή σε πεζά γράμματα
- **Stemming/Lemmatization**: Μείωση λέξεων στη ρίζα τους
- **Stop Words Removal**: Αφαίρεση κοινών λέξεων (άρθρα, σύνδεσμοι)
- **N-grams**: Χρήση bigrams ή trigrams για σύλληψη φράσεων

### Βελτιστοποίηση Παραμέτρων:
- **max_features**: Δοκιμή διαφορετικών τιμών (1000, 5000, 10000)
- **min_df/max_df**: Φιλτράρισμα σπάνιων ή πολύ κοινών όρων
- **SVM C parameter**: Ρύθμιση regularization
- **Cross-validation**: Χρήση K-fold για πιο αξιόπιστα αποτελέσματα

### Αξιολόγηση Αποτελεσμάτων:
- **Confusion Matrix**: Ανάλυση λαθών ανά κατηγορία
- **Feature Importance**: Εύρεση των πιο σημαντικών όρων
- **Learning Curves**: Έλεγχος για overfitting/underfitting